## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score

In [2]:
data = pd.read_csv('/datasets/insurance.csv')

In [3]:
display(data.head())
display(data.info())
display(data.describe())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [4]:
data['Пол'].value_counts()

0    2505
1    2495
Name: Пол, dtype: int64

In [5]:
data['Члены семьи'].value_counts()

1    1814
0    1513
2    1071
3     439
4     124
5      32
6       7
Name: Члены семьи, dtype: int64

In [6]:
data['Возраст'] = data['Возраст'].astype(int)
data['Зарплата'] = data['Зарплата'].astype(int)

In [7]:
display(data.head())
display(data.info())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41,49600,1,0
1,0,46,38000,1,1
2,0,29,21000,0,0
3,0,21,41700,2,0
4,1,28,26100,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Пол                5000 non-null   int64
 1   Возраст            5000 non-null   int64
 2   Зарплата           5000 non-null   int64
 3   Члены семьи        5000 non-null   int64
 4   Страховые выплаты  5000 non-null   int64
dtypes: int64(5)
memory usage: 195.4 KB


None

**Вывод:**
В датасете 5000 строк и пять столбцов. Пропусков нет. Произвели замену типа данных на количественные в столбцах "Возраст" и "Зарплата".

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

In [8]:
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']

In [9]:
class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0
    
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print("R2", r2_score(target, predictions))

R2 0.42494550308169177


**Ответ:** Метрика R2 = 0.42494550308169177. 

**Обоснование:** Если перемножать фичи на коэффициент, то на результат это не повлияет

## Алгоритм преобразования

**Алгоритм**

Умножим наши признаки на случайно взятую обратимую матрицу ($X*P$) и получим новые признаки, при обучении которых появится новый вектор предсказаний $a'$. Проверим, что $a'$ не будет сильно отличаться от изначальных предсказаний $a$:

$E$ - единичная матрица
$$
E = XX^{-1}
$$
$$
Ew = w
$$

Преобразуем предсказание:

$$
a = Xw \longrightarrow XEw \longrightarrow XPP^{-1}w \longrightarrow (XP)w' = a'
$$

Преобразуем формулу обучения, перемножив на рандомную обратную матрицу $Р$:

$$
w = (X^T X)^{-1} X^T y \longrightarrow 
$$
$$
w' = ((XP)^T XP)^{-1} (XP)^T y \longrightarrow
$$
$$
w' = (P^TX^T XP)^{-1} P^TX^T y
$$

Подставляем $w'$ в формулу $a'$:
$$
a' = XP(X^TP^T XP)^{-1} P^TX^T y \longrightarrow
$$
$$
a' = XP(P^{-1}(X^TX)^{-1}P^{T^{-1}})P^TX^Ty \longrightarrow
$$
$$
a' = XE(X^TX)^{-1}EX^Ty \longrightarrow
$$
$$
a' = X(X^TX)^{-1}X^Ty
$$

**Обоснование**

Преобразование показало, что при дополнительном умножении признаков на случайно взятую матрицу, получаем те же предсказания. Это означает, что мы можем кодировать данные клиентов, без риска ухудшения модели.

## Проверка алгоритма

In [10]:
# Умножаем фичи на рандомную матрицу
features1 = np.dot(features, np.random.randint(10, size=(4,4)))
features1

array([[148889, 397051,  99324, 297978],
       [114096, 304278,  76138, 228422],
       [ 63058, 168174,  42087, 126261],
       ...,
       [101748, 271324,  67860, 203596],
       [ 98159, 261741,  65467, 196423],
       [121863, 324973,  81285, 243861]])

In [11]:
model1 = LinearRegression()
model1.fit(features1, target)
predictions1 = model1.predict(features1)
print("R2", r2_score(target, predictions1))

R2 0.4249455030816396


## Общий вывод

При сравнении модели Линейной регрессии при стандартном наборе признаков и модели с признаками, перемноженными на коэффициент, в виде рандомной обратной матрице, получили метрику R2 одинаковой (или приблизительно равной). Это означает, что мы можем зашифровывать данные клиентов без потери качества модели обучения.